In [1]:
import pandas as pd
import ee
import geemap
import ipyleaflet # to show the map in the notebook

## Packages required for using Dynamic World in a Jupyter Notebook

### How to install the EarthEngine Python package?

`EarthEngine`: 

Installation Tutorial: https://developers.google.com/earth-engine/guides/python_install-conda#mac_6

This tutorial requires the installation of Google Cloud: using the following command:
    
```bash
curl https://sdk.cloud.google.com | bash
```

To authenticate, you should: `earthengine authenticate` as the ee.Authenticate() function is not working. (it only works in Google Colab Notebooks)


### How to authenticate? 
Instructions are here: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup


### How to install the earthengine command-line tool in the Google Cloud Shell:
        
```bash
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py && python get-pip.py --user
pip install earthengine-api --upgrade --user
echo 'export PATH="$HOME/.local/bin:$PATH"' >> ~/.bashrc
source ~/.bashrc
```

You can validate this installation by using `earthengine authenticate` in the Google Cloud Shell.
            
- `GeeMap`: 


In [2]:
ee.Initialize(project="jpmorgancapstone")

In [3]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

In [4]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-96.8745,29.0542,-94.1144,30.6785)

Map.centerObject(region)

In [5]:
# Extract Dynamic World Data for year 2019
start_date = '2019-01-01'
end_date = '2020-01-01'

In [6]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

In [7]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='probability')
Map.addLayer(landcover, {}, 'Land Cover')

In [8]:
landcover

# plot the landcover image
# print(landcover)

In [9]:
# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(center=[29.86635000086944, -95.49445], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [10]:
landcover

## Test number 2

In [11]:
import ee
import folium
from IPython.display import Image


In [12]:
ee.Initialize(project="jpmorgancapstone")

In [13]:
region = ee.Geometry.BBox(-96.8745, 29.0542, -94.1144, 30.6785)
year = 2019


In [30]:
startDate = '2019-01-01'
endDate = '2020-01-01'

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
             .filterDate(startDate, endDate) \
             .filterBounds(region) \
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35)) 

#### Why do we get 648 images?

Yes, I understand why this query returns 648 images. The Sentinel-2 satellite system consists of two satellites (Sentinel-2A and Sentinel-2B), which together provide a revisit time of 5 days at the equator. The query you provided filters the Sentinel-2 Harmonized dataset for images within a specified date range (from January 1, 2019, to January 1, 2020) and region, and with a cloud cover percentage less than 35%.

In the given region and time period, 648 images satisfy these conditions. It is important to note that Sentinel-2 images are divided into granules or tiles, each covering a specific area on the Earth's surface. The query returns all the images of the specified region, which may include overlapping or adjacent granules. The total number of images returned may vary depending on the size of the region, the time period, and the cloud cover conditions.

In [ ]:
# Now aggregated (taking the mean)
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
             .filterDate(startDate, endDate) \
             .filterBounds(region) \
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35)) \
             .mean()

In [32]:
s2

In [23]:
built_label = dataset.select("fnf_prob")


In [24]:
built_label

In [25]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data &copy; Google Earth Engine",
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


In [26]:
# Set visualization parameters
vis_params = {
    'min': 0,
    'max': 100,
    'palette': ['red', 'yellow', 'green']
}

# Create a folium map object
my_map = folium.Map(location=[29.8, -95.5], zoom_start=8)

# Add the built_label layer to the map
my_map.add_ee_layer(built_label, vis_params, 'Built Probability')

# Add a layer control panel to the map
folium.LayerControl().add_to(my_map)

# Display the map
my_map

EEException: ImageCollection.load: ImageCollection asset 'ESA/CCI-LC/LC' not found.